# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [2]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files('data/song_data')

In [6]:
# song_files

In [7]:
song_files[0]

'/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json'

In [16]:
filepath = song_files

In [ ]:
pd.

In [17]:
df = pd.read_json(filepath, typ='series')
df = df.to_frame().T

ValueError: Invalid file path or buffer object type: <class 'list'>

In [14]:
df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,None,None,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.932,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [11]:
song_data = df.loc[0,['song_id', 'title', 'artist_id', 'year', 'duration']]
song_data = song_data.values.tolist()

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [12]:
try:
    cur.execute(song_table_insert, song_data)
    conn.commit()
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

Error: Inserting Rows
duplicate key value violates unique constraint "songs_pkey"
DETAIL:  Key (song_id)=(SOMZWCG12A8C13C480) already exists.



Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [12]:
artist_data = df.loc[0,['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
artist_data = artist_data.values.tolist()

In [13]:
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', None, None]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [14]:
artist_table_insert

'\nINSERT INTO artists (artist_id, artist_name, artist_location, artist_latitude, artist_longitude)\nVALUES (%s, %s, %s, %s, %s)\n'

In [15]:
try: 
    cur.execute(artist_table_insert, artist_data)
    conn.commit()
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [16]:
log_files = get_files('data/log_data')

In [17]:
# log_files

In [18]:
filepath = log_files[0]

In [19]:
filepath

'/home/workspace/data/log_data/2018/11/2018-11-30-events.json'

In [20]:
df = pd.read_json(filepath, lines=True)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 18 columns):
artist           330 non-null object
auth             388 non-null object
firstName        376 non-null object
gender           376 non-null object
itemInSession    388 non-null int64
lastName         376 non-null object
length           330 non-null float64
level            388 non-null object
location         376 non-null object
method           388 non-null object
page             388 non-null object
registration     376 non-null float64
sessionId        388 non-null int64
song             330 non-null object
status           388 non-null int64
ts               388 non-null int64
userAgent        376 non-null object
userId           388 non-null object
dtypes: float64(2), int64(4), object(12)
memory usage: 54.6+ KB


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [22]:
# df.page == 'NextSong'

In [23]:
df = df[df.page == 'NextSong']
# df.info()

In [45]:
# df.head()

In [25]:
df['datetime'] = pd.to_datetime(df['ts'], unit='ms')

In [43]:
t = pd.to_datetime(df['ts'])

In [26]:
# df['datetime'].dt.date

In [27]:
# dir(df['datetime'].dt)

In [28]:
time_data = []
time_data.append(df['ts'])
time_data.append(df['datetime'].dt.hour)
time_data.append(df['datetime'].dt.day)
time_data.append(df['datetime'].dt.weekofyear)
time_data.append(df['datetime'].dt.month)
time_data.append(df['datetime'].dt.year)
time_data.append(df['datetime'].dt.weekday)

In [50]:
time_data_test = []
time_data_test.extend((df['ts'],
                       df['datetime'].dt.hour,
                       df['datetime'].dt.day,
                       df['datetime'].dt.weekofyear,
                       df['datetime'].dt.month,
                       df['datetime'].dt.year,
                       df['datetime'].dt.weekday))

In [29]:
column_labels = ['timestamp','hour', 'day', 'week', 'month', 'year', 'weekday']

In [30]:
time_dict = {}
for k, v in zip(column_labels, time_data):
    time_dict[k] = v

In [31]:
time_df = pd.DataFrame(time_dict)
time_df.head()

,timestamp,hour,day,week,month,year,weekday
0,1543537327796,0,1,1,1,1970,3
1,1543540121796,0,1,1,1,1970,3
2,1543540368796,0,1,1,1,1970,3
3,1543540625796,0,1,1,1,1970,3
4,1543540856796,0,1,1,1,1970,3


In [32]:
for i in time_df.iterrows():
    print(i)
    break

(0, timestamp    1543537327796
hour                     0
day                      1
week                     1
month                    1
year                  1970
weekday                  3
Name: 0, dtype: int64)


In [33]:
list(time_df.iterrows())[0]

(0, timestamp    1543537327796
 hour                     0
 day                      1
 week                     1
 month                    1
 year                  1970
 weekday                  3
 Name: 0, dtype: int64)

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [34]:
time_table_insert

'\nINSERT INTO time (start_time, hour, day, week, month, year, weekday)\nVALUES (%s, %s, %s, %s, %s, %s, %s)\n'

In [35]:
try:
    for i, row in time_df.iterrows():
        cur.execute(time_table_insert, list(row))
        conn.commit()
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [36]:
df.sample(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,datetime
311,Philip Glass,Logged In,Rylan,M,23,George,368.40444,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,Knee 2,200,1543595117796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16,1970-01-01 00:25:43.595117796
6,DJ Dizzy,Logged In,Jacob,M,5,Klein,221.15220,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Sexy Bitch,200,1543541299796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,1970-01-01 00:25:43.541299796


In [37]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.columns = ['user_id', 'first_name', 'last_name', 'gender', 'level']
user_df = user_df.drop_duplicates()
user_df.sample(5)

,user_id,first_name,last_name,gender,level
205,85,Kinsley,Young,F,paid
30,24,Layla,Griffin,F,paid
243,16,Rylan,George,M,paid
177,12,Austin,Rosales,M,free
387,5,Elijah,Davis,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [38]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [39]:
# columns : artist, length, song

In [40]:
# try:
#     for index, row in df.iterrows():
#         # get songid and artistid from song and artist tables
#         cur.execute(song_select, (row.song, row.artist, row.length))
#         results = cur.fetchone()
#         print(results)
# except psycopg2.Error as e: 
#     print("Error: Inserting Rows")
#     print (e)

In [48]:
for index, row in df.iterrows():
    print(index)
    print(row)
    break

0
artist                                               Stephen Lynch
auth                                                     Logged In
firstName                                                   Jayden
gender                                                           M
itemInSession                                                    0
lastName                                                      Bell
length                                                     182.857
level                                                         free
location                           Dallas-Fort Worth-Arlington, TX
method                                                         PUT
page                                                      NextSong
registration                                           1.54099e+12
sessionId                                                      829
song                                             Jim Henson's Dead
status                                                      

In [41]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    # timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent 
    songplay_data = (index, row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [42]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.